<a href="https://colab.research.google.com/github/Gail529/Taylor/blob/main/emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#preprocessing the lyrics data
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re 
import string
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from tqdm import tqdm_notebook as tqdm 
from tqdm import trange
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
taylor_lyrics

,0,1
0,State of Grace,"[['I', ""'m"", 'walking', 'fast', 'through', 'th..."
1,Red,"[['Loving', 'him', 'is', 'like', 'Driving', 'a..."
2,Treacherous,"[['Put', 'your', 'lips', 'close', 'to', 'mine'..."
3,I Knew You Were Trouble,"[['Once', 'upon', 'a', 'time', 'A', 'few', 'mi..."
4,All Too Well,"[['I', 'walked', 'through', 'the', 'door', 'wi..."
...,...,...
90,the lakes [Bonus Track],"[['Is', 'it', 'romantic', 'how', 'all', 'of', ..."
91,the last great american dynasty,"[['Rebekah', 'rode', 'up', 'on', 'the', 'after..."
92,this is me trying,"[['I', ""'ve"", 'been', 'having', 'a', 'hard', '..."
93,tolerate it,"[['I', 'sit', 'and', 'watch', 'you', 'reading'..."


In [ ]:
for i in range(0,4):
    rows=taylor_lyrics.iloc[i,0]
    print(rows)

State of Grace
Red
Treacherous
I Knew You Were Trouble


In [ ]:
#preprocessing all the songs
for i in range(0,94):
    song=taylor_lyrics.iloc[i,1]
    text=re.sub(r'\d+','',song)#removing numbers
    #lowercasing
    text_lower=text.lower()
    #punctuation removal 
    def  remove_punctuation(word): #function will take in the lowercased text and then remove all the punctuation
        translator=str.maketrans('','',string.punctuation)
        text_punct=word.translate(translator)
        return text_punct
    result=remove_punctuation(text_lower) #storing the resulting body of  text in variable result
    #removing stopwords 
    def remove_stopwords(word):#The function will   tokenize the result and then filter off  the stop_words 
        stop_words=set(stopwords.words("english")) 
        word_tokens= word_tokenize(word)
        result=[word for word in word_tokens if word not in stop_words ]
        return result
    lyrics=[remove_stopwords(result)]
    #replacing the values of the lyrics column in the dataframe with the  preprocessed  lyrics
    taylor_lyrics.iat[i,1]=lyrics


In [ ]:
taylor_lyrics.to_csv('clean_lyrics.csv')

In [ ]:
data=pd.read_csv('/content/clean_lyrics.csv')
data.head()

,Unnamed: 0,0,1
0,0,State of Grace,"[['walking', 'fast', 'traffic', 'lights', 'bus..."
1,1,Red,"[['loving', 'like', 'driving', 'new', 'maserat..."
2,2,Treacherous,"[['put', 'lips', 'close', 'mine', 'long', 'nt'..."
3,3,I Knew You Were Trouble,"[['upon', 'time', 'mistakes', 'ago', 'sights',..."
4,4,All Too Well,"[['walked', 'door', 'air', 'cold', 'something'..."


In [ ]:
from tqdm import tqdm_notebook as tqdm 
from tqdm import trange
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import string


In [ ]:
emo_lex=pd.read_excel('/content/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx') # emotion lexicon 

In [ ]:
emolex_df=emo_lex[['English (en)','Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']]
emotions=emolex_df.columns.drop('English (en)')
emolex_df.rename(columns={'English (en)':'word'},inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
import re
for i in range(94):
    song=data.iloc[i,2]
    lyrics=re.sub('['+string.punctuation+']','',song)# removing punctuations
    data.iat[i,2]=lyrics


In [ ]:
data.head()

,Unnamed: 0,0,1
0,0,State of Grace,walking fast traffic lights busy streets busy ...
1,1,Red,loving like driving new maserati deadend stree...
2,2,Treacherous,put lips close mine long nt touch focus eye ey...
3,3,I Knew You Were Trouble,upon time mistakes ago sights got alone found ...
4,4,All Too Well,walked door air cold something bout felt like ...


In [ ]:
#the function passes the column(col) of the respective dataframe passed as arguements
def text_emotion(song_df,col):
  new_song_df=song_df.copy()
  #new  dataframe that will store the emotion scores
  emo_df=pd.DataFrame(0,index=new_song_df.index, columns=emotions)
  lemmatizer=WordNetLemmatizer()

  
  with tqdm(total=len(list(new_song_df.iterrows()))) as pbar:
    for i ,row in new_song_df.iterrows(): 
        pbar.update(1) #update the progress bar
        song=word_tokenize(new_song_df.loc[i][col]) #the body of text for each individual song(row)

        for word in song:
            new=lemmatizer.lemmatize(word)#lemmatizing the words
            emo_score=emolex_df[emolex_df.word == new]     
            if not emo_score.empty:
                for emotion in list(emotions):
                    emo_df.at[i,emotion] += emo_score[emotion]


    new_song_df=pd.concat([new_song_df,emo_df] ,axis=1)
    return new_song_df

In [ ]:
final_df=text_emotion(data,'1')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [ ]:
final_df.head(10)

,Unnamed: 0,song-title,lyrics,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,0,State of Grace,walking fast traffic lights busy streets busy ...,21,19,8,11,2,12,11,8,7,8
1,1,Red,loving like driving new maserati deadend stree...,14,12,6,2,2,7,11,17,2,10
2,2,Treacherous,put lips close mine long nt touch focus eye ey...,11,10,10,5,7,10,4,0,2,4
3,3,I Knew You Were Trouble,upon time mistakes ago sights got alone found ...,5,17,3,3,7,8,3,9,2,6
4,4,All Too Well,walked door air cold something bout felt like ...,8,15,8,7,4,3,6,8,3,8
5,5,22,feels like perfect night dress like hipsters m...,14,13,9,13,9,8,11,10,5,7
6,6,I Almost Do,bet time night still bet tired long hard week ...,5,12,3,13,2,6,4,5,4,5
7,7,We Are Never Ever Getting Back Together [Album...,remember broke first time saying enough cause ...,27,5,3,4,2,6,12,3,1,11
8,8,Stay Stay Stay,pretty sure almost broke last night threw phon...,16,22,18,22,16,21,12,20,3,7
9,9,The Last Time,find door like times sure got roads lead imagi...,8,5,1,35,0,4,0,3,6,6


In [ ]:
cols=final_df.columns
col=cols[0]
final_df.rename(columns={'0':'song-title','1':'lyrics'},inplace=True)

In [ ]:
final_df.to_csv('taylor_emotion_scores.csv')

# Join Emotion_df to audio_features 

In [2]:
emo_df=pd.read_csv('/content/taylor_emotion_scores.csv')
emo_df.head(5)

,Unnamed: 0,Unnamed: 0.1,song-title,lyrics,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,0,0,State of Grace,walking fast traffic lights busy streets busy ...,21,19,8,11,2,12,11,8,7,8
1,1,1,Red,loving like driving new maserati deadend stree...,14,12,6,2,2,7,11,17,2,10
2,2,2,Treacherous,put lips close mine long nt touch focus eye ey...,11,10,10,5,7,10,4,0,2,4
3,3,3,I Knew You Were Trouble,upon time mistakes ago sights got alone found ...,5,17,3,3,7,8,3,9,2,6
4,4,4,All Too Well,walked door air cold something bout felt like ...,8,15,8,7,4,3,6,8,3,8


In [3]:
audio_df=pd.read_excel('/content/audio_features.xlsx')
audio_df.head(5)

,album,song,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,evermore,willow,0.833,0.392,0.574,0.001790,0.1450,-9.195,0.1700,81.112,0.529,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,evermore,champagne problems,0.920,0.462,0.240,0.000000,0.1130,-12.077,0.0377,171.319,0.320,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,evermore,gold rush,0.830,0.512,0.462,0.166000,0.1210,-10.491,0.0408,112.050,0.353,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,evermore,â€˜tis the damn season,0.735,0.575,0.434,0.000066,0.1050,-8.193,0.0312,145.916,0.348,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,evermore,tolerate it,0.878,0.316,0.361,0.000027,0.0797,-10.381,0.0488,74.952,0.221,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
audio_df.dropna(axis=1)

,album,song,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,evermore,willow,0.83300,0.392,0.574,0.001790,0.1450,-9.195,0.1700,81.112,0.529,92
1,evermore,champagne problems,0.92000,0.462,0.240,0.000000,0.1130,-12.077,0.0377,171.319,0.320,89
2,evermore,gold rush,0.83000,0.512,0.462,0.166000,0.1210,-10.491,0.0408,112.050,0.353,87
3,evermore,â€˜tis the damn season,0.73500,0.575,0.434,0.000066,0.1050,-8.193,0.0312,145.916,0.348,87
4,evermore,tolerate it,0.87800,0.316,0.361,0.000027,0.0797,-10.381,0.0488,74.952,0.221,86
...,...,...,...,...,...,...,...,...,...,...,...,...
79,Red,Treacherous,0.10100,0.704,0.621,0.000004,0.1400,-8.086,0.0334,109.995,0.395,39
80,Red,I Knew You Were Trouble.,0.00476,0.536,0.464,0.000003,0.0357,-6.761,0.0412,77.318,0.665,48
81,Red,All Too Well,0.04480,0.602,0.605,0.000334,0.1280,-7.803,0.0244,93.061,0.316,48
82,Red,22,0.00254,0.660,0.733,0.001460,0.0473,-6.564,0.0379,104.009,0.680,47


In [5]:
emo_df.rename(columns={'song-title':'song'} ,inplace=True)

In [6]:
song_df=pd.merge(emo_df,audio_df,on='song')

In [7]:
song_df.dropna(axis=1 , inplace=True)

In [8]:
cols=song_df.columns
print(cols)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'song', 'lyrics', 'Positive', 'Negative',
       'Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness',
       'Surprise', 'Trust', 'album', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'popularity'],
      dtype='object')


In [9]:
del song_df['Unnamed: 0']
del song_df['Unnamed: 0.1']
del song_df['lyrics']
del song_df['instrumentalness']


In [10]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 63
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   song          64 non-null     object 
 1   Positive      64 non-null     int64  
 2   Negative      64 non-null     int64  
 3   Anger         64 non-null     int64  
 4   Anticipation  64 non-null     int64  
 5   Disgust       64 non-null     int64  
 6   Fear          64 non-null     int64  
 7   Joy           64 non-null     int64  
 8   Sadness       64 non-null     int64  
 9   Surprise      64 non-null     int64  
 10  Trust         64 non-null     int64  
 11  album         64 non-null     object 
 12  acousticness  64 non-null     float64
 13  danceability  64 non-null     float64
 14  energy        64 non-null     float64
 15  liveness      64 non-null     float64
 16  loudness      64 non-null     float64
 17  speechiness   64 non-null     float64
 18  tempo         64 non-null     fl

In [11]:
song_df.to_csv('song_df.csv')